In [ ]:
import socket
from concurrent.futures import ThreadPoolExecutor
import ipaddress

def scan_port(ip, port):
    """ตรวจสอบพอร์ตของ IP ว่าเปิดหรือไม่"""
    try:
        with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
            s.settimeout(1)
            s.connect((ip, port))
            return port
    except:
        return None

def scan_ip(ip, ports):
    """สแกนพอร์ตของ IP"""
    open_ports = []
    for port in ports:
        if (result := scan_port(ip, port)):
            open_ports.append(result)
    return ip, open_ports

def network_port_scan(network_range, port_range, output_file):
    """สแกนทั้งวง IP และพอร์ต พร้อมบันทึกผลลัพธ์"""
    print(f"Scanning network: {network_range}")
    network = ipaddress.ip_network(network_range, strict=False)
    ports = list(port_range)

    results = []
    with ThreadPoolExecutor(max_workers=100) as executor:
        futures = [executor.submit(scan_ip, str(ip), ports) for ip in network.hosts()]
        for future in futures:
            ip, open_ports = future.result()
            if open_ports:
                results.append((ip, open_ports))

    # บันทึกผลลัพธ์ลงไฟล์
    with open(output_file, "w") as f:
        if results:
            f.write("Scan Results:\n")
            for ip, open_ports in results:
                f.write(f"{ip} -> Open Ports: {open_ports}\n")
            print(f"\nResults saved to {output_file}")
        else:
            f.write("No open ports found in the network.\n")
            print(f"\nNo open ports found. Results saved to {output_file}")

if __name__ == "__main__":
    network_range = "192.168.60.0/24"  # ช่วงเครือข่ายที่ต้องการสแกน
    port_range = range(1, 1025)       # ช่วงพอร์ตที่ต้องการสแกน
    output_file = "scan_results.txt"  # ชื่อไฟล์สำหรับบันทึกผลลัพธ์
    network_port_scan(network_range, port_range, output_file)


Scanning network: 192.168.113.0/24


In [ ]:
from scapy.all import ARP, Ether, srp
import ipaddress

def network_scan(network_range):
    # สร้าง IP Range
    try:
        ip_net = ipaddress.ip_network(network_range, strict=False)
    except ValueError as e:
        print(f"Invalid network range: {e}")
        return

    # เตรียม Packet ARP เพื่อสแกน
    arp = ARP(pdst=str(ip_net))
    ether = Ether(dst="ff:ff:ff:ff:ff:ff")
    packet = ether / arp

    print(f"Scanning network: {network_range}")

    # ส่ง Packet ARP และรับคำตอบ
    result = srp(packet, timeout=2, verbose=0)[0]

    devices = []
    for sent, received in result:
        devices.append({'ip': received.psrc, 'mac': received.hwsrc})

    if devices:
        print("\nAvailable devices in the network:")
        print("IP Address\t\tMAC Address")
        print("-" * 40)
        for device in devices:
            print(f"{device['ip']}\t\t{device['mac']}")
    else:
        print("No devices found on the network.")

if __name__ == "__main__":
    # ตัวอย่างการใช้งาน
    network_range = "192.168.60.0/24"  # เปลี่ยนเป็นวง IP ที่คุณใช้งาน
    network_scan(network_range)
